In [ ]:
using LightGraphs, Random, Distributions, Plots

In [ ]:
Random.seed!(123)

In [ ]:
num_nodes = 1000 # number of individuals in the population
mean_degree = 10 # average number of social connections per individual
transmission_rate = 0.1 # probability of infection transmission per contact
infectious_period = 10 # duration of infectiousness in time steps


In [ ]:
g = watts_strogatz(num_nodes, mean_degree, 0.1)

In [ ]:
infected = rand(1:num_nodes)


In [ ]:
susceptible = fill(true, num_nodes)
infected_nodes = [infected]
recovered_nodes = []


In [ ]:
num_steps = 100

for step in 1:num_steps
    # Infect susceptible individuals who are in contact with infected individuals
    for node in infected_nodes
        neighbors = neighbors(g, node)
        for neighbor in neighbors
            if rand() < transmission_rate && susceptible[neighbor]
                infected_nodes = union(infected_nodes, [neighbor])
                susceptible[neighbor] = false
            end
        end
    end

    # Recover individuals who have been infectious for the duration of the infectious period
    new_recovered_nodes = []
    for node in infected_nodes
        if step - node >= infectious_period
            new_recovered_nodes = union(new_recovered_nodes, [node])
            infected_nodes = setdiff(infected_nodes, new_recovered_nodes)
            recovered_nodes = union(recovered_nodes, new_recovered_nodes)
            susceptible[node] = false
        end
    end
end


In [ ]:
x = [rand() for i in 1:num_nodes]
y = [rand() for i in 1:num_nodes]

scatter(x[susceptible], y[susceptible], label="Susceptible")
scatter!(x[infected_nodes], y[infected_nodes], label="Infected")
scatter!(x[recovered_nodes], y[recovered_nodes], label="Recovered")


In [1]:
using LightGraphs, Random, Distributions, Plots

Random.seed!(123)

function simulate_sir(num_nodes::Int, mean_degree::Int, transmission_rate::Float64, infectious_period::Int, num_steps::Int)
    g = watts_strogatz(num_nodes, mean_degree, 0.1)

    infected = rand(1:num_nodes)

    susceptible = fill(true, num_nodes)
    infected_nodes = [infected]
    recovered_nodes = []

    function neighbors(g, node)
        out_edges = outedges(g, node)
        in_edges = inedges(g, node)
        neighbors = union([e.dst for e in out_edges], [e.src for e in in_edges])
        return neighbors
    end

    for step in 1:num_steps
        for node in infected_nodes
            neighbors = neighbors(g, node)
            for neighbor in neighbors
                if rand() < transmission_rate && susceptible[neighbor]
                    infected_nodes = union(infected_nodes, [neighbor])
                    susceptible[neighbor] = false
                end
            end
        end

        new_recovered_nodes = []
        for node in infected_nodes
            if step - node >= infectious_period
                new_recovered_nodes = union(new_recovered_nodes, [node])
                infected_nodes = setdiff(infected_nodes, new_recovered_nodes)
                recovered_nodes = union(recovered_nodes, new_recovered_nodes)
                susceptible[node] = false
            end
        end
    end

    x = [rand() for i in 1:num_nodes]
    y = [rand() for i in 1:num_nodes]

    scatter(x[susceptible], y[susceptible], label="Susceptible")
    scatter!(x[infected_nodes], y[infected_nodes], label="Infected")
    scatter!(x[recovered_nodes], y[recovered_nodes], label="Recovered")
end

simulate_sir(1000, 10, 0.1, 10, 100)


UndefVarError: UndefVarError: outedges not defined

In [3]:
using CSV, DataFrames, DifferentialEquations, Plots

# Load the John Hopkins daily confirmed COVID-19 cases data
df_cases = CSV.read(download("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"), DataFrame)
countries = ["United Kingdom", "Italy", "Spain", "Germany", "France"] # Select a few countries
df_cases = df_cases[df_cases[!, "Country/Region"] .∈ countries, :]
df_cases = by(df_cases, "Country/Region", select=collect(5:size(df_cases, 2)) .=> sum)
df_cases = stack(df_cases, names(df_cases, Not(:Country_Region)), variable_name=:date, value_name=:cases)
df_cases.date = Date.(df_cases.date, "m/d/y")
df_cases = filter(row -> row.cases > 0, df_cases)



DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 289 and 5

In [ ]:
# Define the SEIR model
function seir!(du, u, p, t)
    β, σ, γ, v = p
    S, E, I, R, V = u
    N = S + E + I + R
    du[1] = -β*S*I/N
    du[2] = β*S*I/N - σ*E
    du[3] = σ*E - γ*I
    du[4] = γ*I
    du[5] = v
end

# Set the initial conditions and parameters
S0 = 6.7e7 # Population of the UK
E0 = 1.0
I0 = 1.0
R0 = 0.0
V0 = 0.0
u0 = [S0, E0, I0, R0, V0]
β = 0.3 # Infection rate
σ = 0.1 # Incubation rate
γ = 0.05 # Recovery rate
v = 0.0 # Vaccination rate
p = [β, σ, γ, v]

# Set the time span and solve the SEIR model using DifferentialEquations.jl
tspan = (minimum(df_cases.date), maximum(df_cases.date) + Day(30))
prob = ODEProblem(seir!, u0, tspan, p)
sol = DifferentialEquations.solve(prob, Tsit5())

# Compute the daily confirmed cases from the SEIR model
cases_seir = S0 .- sol[!, 1] .- sol[!, 2] .- sol[!, 3] .- sol[!, 4]

# Plot the results
plot(df_cases.date, df_cases.cases, label="Confirmed cases (data)")
plot!(sol.t, cases_seir, label="Confirmed cases (SEIR model)", xlabel="Date", ylabel="Number of cases", legend=:topright)
